### Import and load env

In [ ]:
from dotenv import load_dotenv
import os
import sqlite3
import sys

load_dotenv()

MSG_DB_PATH = os.environ.get("MSG_DB_PATH")
FULL_DB_PATH = os.environ.get("FULL_DB_PATH")
START_TIMESTAMP = os.environ.get("START_TIMESTAMP")
END_TIMESTAMP = os.environ.get("END_TIMESTAMP")
TARGET_ID = os.environ.get("TARGET_ID")
print("load_dotenv():")
print(f"  {MSG_DB_PATH=}")
print(f"  {FULL_DB_PATH=}")
print(f"  {START_TIMESTAMP=}")
print(f"  {END_TIMESTAMP=}")
print(f"  {TARGET_ID=}")

### Define helper methods

In [ ]:
def connect_db(path):
    try:
        conn = sqlite3.connect(path)
        return conn
    except sqlite3.Error as e:
        print(f"Error connecting db: {e}")
        sys.exit(1)

def execute_query(conn, query, params=(), no_commit=False):
    try:
        cursor = conn.cursor()
        cursor.execute(query, params)
        if not no_commit:
            conn.commit()
        if query.strip().lower().startswith("select"):
            return cursor.fetchall()
        else:
            return None
    except sqlite3.Error as e:
        print(f"Error executing SQL query: {e}")
        sys.exit(1)

### Create or load MSG db

In [ ]:
if not os.path.exists(MSG_DB_PATH):
    full_conn = connect_db(FULL_DB_PATH)

    query = f"""
    SELECT *
    FROM MSG
    WHERE Sequence >= {START_TIMESTAMP + '000'}
      AND Sequence <  {END_TIMESTAMP + '000'}
      AND StrTalker = '{TARGET_ID}';
    """
    msg = execute_query(full_conn, query)
    if not msg:
        print("error: no result from FULL_DB_PATH")
        sys.exit(1)
    print(f"Found {len(msg)} results.")
    full_conn.close()
    
    msg_conn = connect_db(MSG_DB_PATH)

    query = """
    CREATE TABLE MSG(
      localId INT,
      TalkerId INT,
      MsgSvrID INT,
      Type INT,
      SubType INT,
      IsSender INT,
      CreateTime INT,
      Sequence INT,
      StatusEx INT,
      FlagEx INT,
      Status INT,
      MsgServerSeq INT,
      MsgSequence INT,
      StrTalker TEXT,
      StrContent TEXT,
      DisplayContent TEXT,
      Reserved0 INT,
      Reserved1 INT,
      Reserved2 INT,
      Reserved3 INT,
      Reserved4 TEXT,
      Reserved5 TEXT,
      Reserved6 TEXT,
      CompressContent,
      BytesExtra,
      BytesTrans
    )
    """
    execute_query(msg_conn, query)
    query = "INSERT INTO MSG VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)"
    for i, row in enumerate(msg):
        execute_query(msg_conn, query, row, no_commit=True)
        if i % 5000 == 0:
            msg_conn.commit()
    msg_conn.commit()
    print(f"{MSG_DB_PATH} is created.")
else:
    msg_conn = connect_db(MSG_DB_PATH)
    print(f"{MSG_DB_PATH} is found and loaded.")